<a href="https://colab.research.google.com/github/DaAnMaGi/Proyecto_Marzo/blob/main/Parte%202/recomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo del recomedación

## Importación de bases

### Importación de librerías

In [1]:
! pip install cython
! pip install wheel
! pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162726 sha256=ba63323fb47a8b8b97f7e1a4e0a73bb9d41d91ba80598c2a919f6df4533acf9e
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [25]:
# Importación de librerías
import pandas as pd
import numpy as np

from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise.model_selection import cross_validate

### Importación de bases de datos

In [4]:
# Este notebook fue trabajado desde Google Collabs, debido a dificultades para installar la librería surprise en el computador, por lo que se presentan rutas de archivo diferente para tratar de cargar las bases.

# Desde la pc
# base = pd.read_parquet("../Data/final/total_generos.parquet")

# Reviews
# reviews = pd.read_parquet("../Data/final/reviews_final.parquet")
# Games
# games = pd.read_parquet("../Data/final/games_final.parquet")
# items
# items = pd.read_parquet("../Data/final/items_final.parquet")

# En Google Colabs
# Reviews
reviews = pd.read_parquet("/content/reviews_final.parquet")
# Games
games = pd.read_parquet("/content/games_final.parquet")
# items
items = pd.read_parquet("/content/items_final.parquet")


## Creación del modelo de recomendación

In [5]:
r = reviews[["user_id","item_id","sentiment_analysis"]]
r

,user_id,item_id,sentiment_analysis
0,76561197970982479,1250,2
1,76561197970982479,22200,1
4,js41637,227300,2
5,js41637,239030,2
6,evcentric,248820,1
...,...,...,...
57629,76561198312638244,233270,2
57630,76561198312638244,130,2
57631,76561198312638244,70,2
57632,76561198312638244,362890,2


In [31]:
reader = Reader(rating_scale=[0,2])

datos = Dataset.load_from_df(r,reader)

In [32]:
modelo = SVD()

In [33]:
cross_validate(modelo,datos,measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6362  0.6323  0.6369  0.6342  0.6411  0.6361  0.0029  
MAE (testset)     0.5528  0.5512  0.5534  0.5528  0.5575  0.5535  0.0021  
Fit time          1.09    1.30    1.20    0.81    0.86    1.05    0.19    
Test time         0.12    0.08    0.05    0.06    0.05    0.07    0.03    


{'test_rmse': array([0.63619558, 0.63232795, 0.63685164, 0.63421155, 0.64107683]),
 'test_mae': array([0.55282402, 0.55117876, 0.55343709, 0.55276242, 0.55752173]),
 'fit_time': (1.0889723300933838,
  1.3026325702667236,
  1.197145700454712,
  0.8111934661865234,
  0.8570713996887207),
 'test_time': (0.11944341659545898,
  0.07741594314575195,
  0.04957938194274902,
  0.056035757064819336,
  0.05406975746154785)}

In [34]:
entrenamiento = datos.build_full_trainset()
modelo.fit(entrenamiento)

In [42]:
def get_recommendations(user_id:str, n=5):
    model = modelo
    # Obtén todos los ítems que el usuario no ha calificado
    items_unrated = [item for item in entrenamiento.all_items() if item not in entrenamiento.ur[user_id]]

    # Haz predicciones para los ítems no calificados por el usuario
    predictions = [model.predict(user_id, item) for item in items_unrated]

    # Ordena las predicciones en orden descendente de calificación
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Devuelve las mejores 'n' recomendaciones
    top_n = predictions[:n]

    return top_n

In [43]:
get_recommendations("js41637")

[Prediction(uid='js41637', iid=440, r_ui=None, est=1.8167447331907067, details={'was_impossible': False}),
 Prediction(uid='js41637', iid=630, r_ui=None, est=1.7886516552820746, details={'was_impossible': False}),
 Prediction(uid='js41637', iid=2600, r_ui=None, est=1.7743509061137097, details={'was_impossible': False}),
 Prediction(uid='js41637', iid=1200, r_ui=None, est=1.7427689183772854, details={'was_impossible': False}),
 Prediction(uid='js41637', iid=2300, r_ui=None, est=1.7417128155271309, details={'was_impossible': False})]